In [1]:
import sys
module_path = "../"
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from model import *
from data import *
from helpers import *
import numpy as np
import pandas as pd
import yaml
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import DataLoader, Dataset
import sys
import time

In [3]:
X = torch.Tensor(pd.read_csv('../../data/DataFiles/Algebra30.csv', header=None).values)
Q = torch.Tensor(pd.read_csv('../QMatrices/QMatrixAlgebra.csv', index_col=0).values)


In [4]:
with open("../config.yml", "r") as f:
    cfg = yaml.safe_load(f)
    cfg = cfg['configs']


In [7]:
best_mean_cor = 0
for i in range(20):
    print(i)
    # initialise model and optimizer
    logger = CSVLogger("logs", name='algebra', version=0)
    trainer = Trainer(fast_dev_run=cfg['single_epoch_test_run'],
                      max_epochs=cfg['max_epochs'],
                      enable_checkpointing=False, 
                      logger=False, 
                      callbacks=[EarlyStopping(monitor='train_loss', min_delta=cfg['min_delta'], patience=cfg['patience'], mode='min')])
    
    dataset = SimDataset(X)
    train_loader = DataLoader(dataset, batch_size=cfg['batch_size'], shuffle=False)
    vae = CVAE(nitems=X.shape[1],
                dataloader=train_loader,
                latent_dims=cfg['mirt_dim'],
                hidden_layer_size=cfg['hidden_layer_size'],
                qm=Q,
                learning_rate=cfg['learning_rate'],
                batch_size=cfg['batch_size'],
                beta=cfg['beta'],
                n_samples=cfg['n_iw_samples']
              )
    
    start = time.time()
    trainer.fit(vae)
    runtime = time.time()-start
    print(runtime)
    
    a_est = vae.decoder.weights.t().detach().numpy()
    d_est = vae.decoder.bias.t().detach().numpy()

    cors = np.array([])
    for d in range(9):
        if np.corrcoef(a_est[:,d], a_est_full[:,d])[1,0] < 0:
            a_est[:,d] *= -1
        
        cors = np.append(cors, np.corrcoef(a_est[:,d], a_est_full[:,d])[1,0])

    meancor = cors.mean()

    if meancor > best_mean_cor:
        best_a_est = a_est
        best_d_est = d_est
        best_vae = vae



0


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karel/Documents/GitHub/VAE-MIRT-Missing/MIRTVAE/additional_scripts/../data.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x_train = torch.tensor(X, dtype=torch.float32)

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)
/Library/Frameworks/Python.framework/Versions/3.10/lib/pyth

{'nitems': 75, 'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x168245f00>, 'latent_dims': 9, 'hidden_layer_size': 20, 'qm': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


{'nitems': 75, 'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x2b6d6c5e0>, 'latent_dims': 9, 'hidden_layer_size': 20, 'qm': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0

...

Epoch 48: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 69.78it/s]
29.364980936050415
17


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


{'nitems': 75, 'dataloader': <torch.utils.data.dataloader.DataLoader object at 0x2b6ca6050>, 'latent_dims': 9, 'hidden_layer_size': 20, 'qm': tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 1., 0., 0., 0., 0., 1., 0., 1.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 1., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 74: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 69.14it/s]
45.12769603729248


In [8]:
a_est = best_a_est
d_est= best_d_est
vae = best_vae

In [ ]:
cors

In [9]:
dataset = SimDataset(X)
train_loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False)
data, mask = next(iter(train_loader))
_, log_sigma_est  = vae.encoder(data, mask)
post_samples = vae.fscores((data, mask), cfg['model'])
theta_est = post_samples.mean(0)


In [10]:
mu = theta_est.repeat(500, 1, 1)
sigma = log_sigma_est.repeat(500, 1, 1)
vae.transform.n_samples = 500
z = vae.sampler(mu, sigma)
z_transformed = vae.transform(z)



# For each of the 10000 elements, we need to compute the corss products
cov_matrix = torch.einsum('ijk,ijl->jkl', z_transformed, z_transformed) / (z_transformed.shape[0] - 1)

# Step 4: Convert the covariance matrix to a correlation matrix
std_dev = torch.sqrt(torch.diagonal(cov_matrix, dim1=-2, dim2=-1)).unsqueeze(-1)
est_cov_mat = (cov_matrix / (std_dev @ std_dev.transpose(-2, -1))).detach().numpy().mean(0)
est_cor_mat = np.abs(cov2cor(est_cov_mat))


In [11]:
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/a_est', a_est, 
              delimiter = ",")
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/d_est', d_est, 
              delimiter = ",")
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/theta_est', theta_est, 
              delimiter = ",")



In [6]:
a_est_full = pd.read_csv('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/a_est_full', index_col=0).values
